In [26]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import re
import requests

In [27]:
proxy = "http://45.140.13.124:9137"

In [67]:
def getdriver(): #it is the selenium driver to connect the script with the browser
    options = webdriver.ChromeOptions()
    options.add_argument('log-level=3')
    options.add_argument("--window-size=1880x1020")
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    #options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("test-type")
    options.add_argument("--disable-web-security")
    options.add_argument("--allow-running-insecure-content")
    options.add_argument('--proxy-server=%s' % proxy)
    driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())
    #     driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """})
    return driver
driver = getdriver()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Pfactorial00110\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\Pfactorial00110\AppData\Local\Temp\ipykernel_6816\3269282774.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())


In [75]:
job_link = []
Category = []

def neuvoo_extractor():#this functions extracts the job links from the particular websie
  try:
    for i in range(1,3): # iterates upto 10 pages
      url_next='https://neuvoo.ca/jobs/?k=data+science&l=&f=&o=&p=&r=15'
      w1=requests.get(url_next,proxy) # to access the url
      soups=BeautifulSoup(w1.content,'html.parser') 
      for item in soups.findAll("h2", itemprop='title'): # iterates through the items which has href links
        for a in item.findAll('a', href=True):
          job_link.append('https://www.neuvoo.com'+a['href']) # adds the iterted data to the empty list
          Category.append('Data Science')
  except:
    pass
  return job_link,Category

In [76]:
def neuvoo_extractor():#this functions extracts the job links from the particular websie
  try:
    for i in range(0,10): # iterates upto 10 pages
      url_next='https://neuvoo.ca/jobs/?k=machine+learning&l=&p='+str(i)+'&date=&field=&company=&source_type=&radius=&from=&test=&iam=&is_category=no'
      driver.get(url_next) # to access the url
      content = driver.page_source 
      soups=BeautifulSoup(content,'html.parser')
      for i in soups.find_all(class_='card card__job'):
        ids = i['dataid']
        Category.append('Machine Learning')
        job_link.append('https://ca.talent.com/view?id='+str(ids)+'&context=geo_bulk_bounce&e=1')
  except:
    pass
  return job_link,Category


In [77]:
job_link,Category = neuvoo_extractor()

In [80]:
Job_posting_df=pd.DataFrame({"Category":Category,"Job_Posting_link":job_link})

In [81]:
Job_posting_df.to_csv('Machine_Learning_neuvoo_links.csv', index=False)

In [83]:
DS_links = pd.read_csv("Data_science_neuvoo_links.csv")
ML_links = pd.read_csv("Machine_Learning_neuvoo_links.csv")

In [84]:
def neuvoo_details_extractor(df): # this function extracts the data from the particular job link
    neuvoo_df = pd.DataFrame()
    for i in range(0,len(df)): #it iterates through the links upto the extracted links length
        try:
            Job_Posting_link=df['Job_Posting_link'][i] #iterates through the links in the dataframe
            Category=df['Category'][i]
            driver.get(Job_Posting_link)
            content = driver.page_source
            soups1=BeautifulSoup(content,'html.parser')
            try:
                Location=soups1.find(class_='job__header__location').text #extracts the text data from the respective class
                pass
            except:
                Location=None
            try:
                company = soups1.find(class_ = 'job__header__company').text
                pass
            except:
                company=None
            try:
                title = soups1.find(class_ = 'job__header__title').text
                pass
            except:
                title=None
            try:
                salary = soups1.find(class_ ='salary').text
                pass
            except:
                salary=None
            try:
                description = soups1.find(class_='job__description').text
            except:
                description=None
            neuvoo=pd.DataFrame({'Category':Category,'Job_Posting_link':Job_Posting_link,'Job_title':title,'Company_name':company,'Location':Location,'Description':[description]},index=[0])
            # print(simplyhired1)
            neuvoo_df=neuvoo_df.append(neuvoo)
        except Exception as e: # raises an exception if something goes wrong in the time of extraction
            print(e)   
    return neuvoo_df

In [91]:
data_science = neuvoo_details_extractor(DS_links) #calls the above funtion

In [92]:
Machine_Learning = neuvoo_details_extractor(ML_links) #calls the above funtion

In [87]:
data_science.to_csv("neuvoo_DS_full_details.csv", index=False)

In [90]:
Machine_Learning.to_csv("neuvoo_ML_full_details.csv", index=False)